In [1]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install simple-salesforce

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.0/761.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 8.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
!python3 -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
env CFLAGS="-I/opt/homebrew/opt/portaudio/include" LDFLAGS="-L/opt/homebrew/opt/portaudio/lib" pip install PyAudio


env: CFLAGS="-I/opt/homebrew/opt/portaudio/include" LDFLAGS="-L/opt/homebrew/opt/portaudio/lib" pip install PyAudio


In [81]:
import speech_recognition as sr
import requests
import spacy

# Função para capturar o áudio
def capture_audio():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Diga algo:")
        recognizer.adjust_for_ambient_noise(source, duration=1)

        try:
            # Ouve com limite de tempo (5 segundos para falar, timeout de 10 segundos)
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=5)

            # Usa o Google Web Speech API para reconhecer o áudio
            text = recognizer.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {text}")
            return text
        
        except sr.WaitTimeoutError:
            print("Tempo esgotado para a fala.")
            return None
        except sr.UnknownValueError:
            print("Desculpe, não consegui entender o áudio.")
            return None
        except sr.RequestError:
            print("Erro ao se conectar ao serviço de reconhecimento de fala.")
            return None

# Função spaCy para processar a frase e identificar as informações
def refine_user_input_spacy(user_input):
    # Carrega o modelo de português do spaCy
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(user_input.lower())  # Converte a frase para minúsculas para facilitar a análise

    entity = None
    name = None
    
    # Verifica se o input contém "contato" ou "conta" e define a entidade
    if "contato" in user_input.lower():
        entity = "Contact"
    elif "conta" in user_input.lower():
        entity = "Account"

    # Extrair o nome que vem depois de "contato" ou "conta"
    for i, token in enumerate(doc):
        if token.text == "contato" or token.text == "conta":
            # Pegamos as palavras seguintes após "contato" ou "conta", assumindo que é o nome
            name_tokens = [t.text for t in doc[i + 1:] if not t.is_punct and not t.is_space]  # Ignorar pontuações e espaços
            name = " ".join(name_tokens).strip()  # Juntar os tokens para formar o nome
            break

    return {"entity": entity, "name": name}

# Função para fazer a busca no Salesforce e montar o "storytelling"
def search_salesforce_info(entity, name):
    # Bearer token atualizado
    bearer_token = '00D6O000000Wc9b!AQ8AQJNzIFSGuhxb7KArroQq_M88fu48DUe_19COrn6Ox0_9m.VRNz59WdgWvt2buC.rQ8_bD67m084e5yj1MByAG9FZlaHm'
    
    # URL do endpoint para buscar dados
    search_url = f'https://orgfarmcom457-dev-ed.develop.my.salesforce.com/services/data/v57.0/query'
    
    # Definindo a query com base na entidade e nome (pesquisa pelo nome)
    query = f"""
    SELECT Name, Description, Email, Phone, AccountId, Title, MailingAddress
    FROM {entity} 
    WHERE Name LIKE '%{name}%'
    """
    
    print(f"Executando a query:\n{query}")  # Para verificar o conteúdo da query
    
    # Headers da requisição com o Bearer Token
    headers = {
        'Authorization': f'Bearer {bearer_token}',
        'Content-Type': 'application/json'
    }
    
    # Fazendo a requisição GET para obter os dados
    try:
        response = requests.get(f"{search_url}?q={query}", headers=headers)
        
        # Verificar a resposta
        if response.status_code == 200:
            # Processar os registros retornados
            records = response.json().get('records', [])
            if records:
                # Pegando o primeiro registro retornado
                record = records[0]
                name = record.get('Name', 'Não disponível')
                description = record.get('Description', 'Nenhuma descrição fornecida')
                email = record.get('Email', 'Email não disponível')
                phone = record.get('Phone', 'Telefone não disponível')
                account_id = record.get('AccountId', 'Não disponível')
                title = record.get('Title', 'Título não disponível')

                # Montando o endereço
                mailing_address = record.get('MailingAddress', {})
                if mailing_address:
                    address = f"{mailing_address.get('street', 'Rua não disponível')}, {mailing_address.get('city', 'Cidade não disponível')}, {mailing_address.get('state', 'Estado não disponível')}"
                else:
                    address = "Endereço não disponível"
                
                # Montagem do "storytelling"
                story = f"{name} é o contato que você pesquisou. "
                story += f"Ele(a) trabalha como {title} e está associado à conta {account_id}. "
                story += f"Você pode entrar em contato com ele(a) pelo e-mail {email} ou pelo telefone {phone}. "
                story += f"Aqui está a descrição: '{description}'. "
                story += f"O endereço dele(a) é: {address}."

                print("\n--- Storytelling ---")
                print(story)
            else:
                print(f"Nenhum registro encontrado para o nome '{name}'")
        else:
            print(f"Erro ao obter dados: {response.status_code}, {response.text}")
    
    except Exception as e:
        print(f"Erro ao fazer a requisição: {str(e)}")

# Captura o áudio e usa o resultado com o spaCy
text_from_audio = capture_audio()

# Se o áudio foi capturado, passa para o spaCy
if text_from_audio:
    result = refine_user_input_spacy(text_from_audio)
    print("Resultado refinado pelo spaCy:", result)

    # Verifica se a entidade e o nome foram encontrados
    if result["entity"] and result["name"]:
        # Faz a busca no Salesforce e monta o storytelling
        search_salesforce_info(result["entity"], result["name"])
    else:
        print("Não foi possível identificar a entidade ou o nome corretamente.")
else:
    print("Nenhum áudio foi capturado.")

Diga algo:
Você disse: traz informações sobre o contato Sofia
Resultado refinado pelo spaCy: {'entity': 'Contact', 'name': 'sofia'}
Executando a query:

    SELECT Name, Description, Email, Phone, AccountId, Title, MailingAddress
    FROM Contact 
    WHERE Name LIKE '%sofia%'
    

--- Storytelling ---
Sofia Rodrigues é o contato que você pesquisou. Ele(a) trabalha como None e está associado à conta 0016O00003vjKmEQAU. Você pode entrar em contato com ele(a) pelo e-mail sofiarodriguez@example.com ou pelo telefone None. Aqui está a descrição: 'Sofia Rodriguez is a Contact with the email sofiarodriguez@example.com and resides at 3 6th Avenue, Minneapolis, Minnesota, United States. Sofia, who prefers to be called Sofi, has interests in *Adventure Activities*, *Beaches & Snorkeling*, and *Fitness & Exercise*. The lead source is Direct Mail, and the guest type is Cultural Connoisseurs. There are no open Tasks or future Events, but there is an email welcoming Sofia to Coral Clouds Resort, se